# 501 Prepare data for global assessment

In this notebook, we prepare the data necessary for our global assessment.

In [7]:
import pyam
import pandas as pd
import numpy as np
import scmdata
from pathlib import Path
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

Step 1: Read in the necessary data.

In [8]:
cdr_original = pyam.IamDataFrame(
    Path(
        '../data/100_novel_cdr.csv'
    )
)

pyam.core - INFO: Reading file ../data/100_novel_cdr.csv


In [9]:
cdr_climate = pyam.IamDataFrame(
    Path(
        '../data/405_cdr_170324.csv'
    )
)

pyam.core - INFO: Reading file ../data/405_cdr_170324.csv


In [10]:
metrics = pd.read_csv(
    Path('../data/401_lookup.csv'),
    index_col=[0,1]
)

Step 2: Now, per ensemble member, we want to calculate the additional cumulative CDR between net zero CO2 and 2100.

In [11]:
additional_cdr_climate = []

In [12]:
cdr_original.interpolate(
    time=range(2015, 2101),
    inplace=True
)

In [13]:
for (model, scenario) in tqdm(cdr_original.meta.index):
    # Filter for original CDR
    cdr_orig = (
        cdr_original
        .filter(
            model=model,
            scenario=scenario
        )
        .timeseries()
    )
    # Filter for adapted CDR
    cdr_alt = (
        cdr_climate
        .filter(
            model=model,
            scenario=scenario,
        )
        .timeseries()
    )
    # Subtract original from adapted
    cdr_diff = pyam.IamDataFrame(cdr_alt - cdr_orig)
    additional_cdr_climate.append(
        pyam.IamDataFrame(cdr_diff)
    )

  0%|          | 0/95 [00:00<?, ?it/s]

In [14]:
additional_cdr_climate = pyam.concat(additional_cdr_climate)

In [15]:
additional_cdr_climate.rename(
    variable={
        'Carbon Dioxide Removal|Novel':'Carbon Dioxide Removal|Novel|Additional'
    },
    inplace=True
)

Step 3: Convert all the necessary units to Gt CO2

In [16]:
additional_cdr_climate.convert_unit(
    current='Mt CO2/yr',
    to='Gt CO2/yr',
    inplace=True
)

In [17]:
cdr_original.convert_unit(
    current='Mt CO2/yr',
    to='Gt CO2/yr',
    inplace=True
)

Step 4: Now, we want to calculate the cumulative additional CDR from net zero CO2 to 2100. We can just sum up across all values for each timeseries entry, since we have already calculated the difference from the original model trajectory.

In [18]:
cumulative_cdr_additional = (
    additional_cdr_climate
    .timeseries()
    .apply(
        lambda x: format(pyam.timeseries.cumulative(
            x,
            first_year=2020,
            last_year=2100
        ),'f'),
        axis=1
    )
)

Step 5: Assign this as an additional column to the metrics dataframe. Then write this out for further analysis.

In [19]:
metrics = metrics.set_index('run_id', append=True)

In [20]:
for (mod, scen, run) in metrics.index:
    if mod == 'GEM-E3_V2021':
        continue
    metrics.loc[(mod, scen, run), 'additional_cdr_gtco2_final'] = (
        cumulative_cdr_additional
        .loc[pd.IndexSlice[mod, scen, :, :, :, :, run]]
        .values[0]
    )

In [21]:
metrics['additional_cdr_gtco2_final'] = metrics['additional_cdr_gtco2_final'].astype('float').round(2)

In [22]:
metrics.to_excel(
    Path(
        '../data/501_metrics_with_final_cdr.xlsx'
    ),
    merge_cells=False
)

In [23]:
metrics

drawdown_co2       ZEC  \
model             scenario             run_id                           
AIM/CGE 2.1       CD-LINKS_NPi2020_400 0           0.153213 -0.086132   
                                       1          -0.025027 -0.016988   
                                       2           0.164106 -0.065352   
                                       3           0.122711 -0.119902   
                                       4           0.143798 -0.097182   
...                                                     ...       ...   
WITCH-GLOBIOM 4.4 CD-LINKS_NPi2020_400 595         0.205725 -0.155350   
                                       596         0.245548 -0.010975   
                                       597         0.143254 -0.004426   
                                       598         0.167401 -0.042595   
                                       599         0.186103 -0.002185   

                                               change_non_co2  2015_warming  \
model             scenario             run_id                                 
AIM/CGE 2.1       CD-LINKS_NPi2020_400 0             0.024094      1.107815   
                                       1             0.054163      1.393747   
                                       2            -0.009230      1.239155   
                                       3            -0.081853      0.983549   
                                       4             0.035314      1.194385   
...                                                       ...           ...   
WITCH-GLOBIOM 4.4 CD-LINKS_NPi2020_400 595          -0.189144      1.088553   
                                       596          -0.053872      1.295575   
                                       597          -0.003011      1.224026   
                                       598          -0.142359      1.045557   
                                       599          -0.093717      1.073903   

                                               2100_warming  peak_warming  \
model             scenario             run_id                               
AIM/CGE 2.1       CD-LINKS_NPi2020_400 0           1.391478      1.614685   
                                       1           2.494802      2.498626   
                                       2           1.542584      1.788063   
                                       3           1.031367      1.357680   
                                       4           1.483014      1.695355   
...                                                     ...           ...   
WITCH-GLOBIOM 4.4 CD-LINKS_NPi2020_400 595         0.818536      1.368591   
                                       596         1.659452      1.918303   
                                       597         1.486971      1.641801   
                                       598         0.993811      1.350887   
                                       599         1.100671      1.374493   

                                               year_peak_warming  \
model             scenario             run_id                      
AIM/CGE 2.1       CD-LINKS_NPi2020_400 0                    2036   
                                       1                    2036   
                                       2                    2036   
                                       3                    2028   
                                       4                    2037   
...                                                          ...   
WITCH-GLOBIOM 4.4 CD-LINKS_NPi2020_400 595                  2060   
                                       596                  2069   
                                       597                  2069   
                                       598                  2069   
                                       599                  2082   

                                               cum_emissions_to_peak_GtCO2  \
model             scenario             run_id                                
AIM/CGE 2.1       CD-LINKS

# Checks!
Here, we do some checks to ensure we're getting sensible outcomes.
1. Make sure that the additional CDR is zero in the time step before the year of net zero CO2.

In [26]:
additional_cdr_climate_ts = (
    additional_cdr_climate
    .timeseries()
)

In [29]:
for (mod, scen) in tqdm(additional_cdr_climate.meta.index):
    for run_id in range(600):
        nz_year = metrics.loc[(mod, scen, run_id), 'netzero|CO2']
        value = (
            additional_cdr_climate
            .filter(
                model=mod,
                scenario=scen,
                year=nz_year - 1,
                ensemble_member=run_id
            )
            .timeseries()
            .values[0]
        )
        assert int(value) == 0

  0%|          | 0/95 [00:00<?, ?it/s]

2. Now, let us sum up the numbers and calculate the 90th percentile for each model-scenario combination

In [39]:
additional_cdr_climate_ts.sum(axis=1).groupby(['model', 'scenario']).quantile(q=0.9).describe().astype('int')

count      95
mean      531
std       286
min         0
25%       268
50%       568
75%       762
max      1127
dtype: int64

Ok, need to change this in the text!

In [42]:
additional_cdr_climate_ts.sum(axis=1).groupby(['model', 'scenario']).quantile(q=0.9).astype(int).head(30)

model                  scenario                
AIM/CGE 2.1            CD-LINKS_NPi2020_400        408
AIM/CGE 2.2            EN_NPi2020_300f             456
                       EN_NPi2020_600              857
AIM/Hub-Global 2.0     1.5C                        395
C-ROADS-5.005          Ratchet-1.5-limCDR-noOS     220
                       Ratchet-1.5-noCDR           684
                       Ratchet-1.5-noCDR-noOS      460
COFFEE 1.1             EN_NPi2020_400              963
GCAM 4.2               SSP1-19                     604
GCAM 5.3               R_MAC_30_n0                 752
                       R_MAC_35_n8                 188
                       R_MAC_40_n8                 298
                       R_MAC_45_n8                 462
                       R_MAC_50_n8                 667
IMAGE 3.2              SSP1_SPA1_19I_D_LB          462
                       SSP1_SPA1_19I_LIRE_LB       335
                       SSP1_SPA1_19I_RE_LB         520
                 